In [ ]:
# Environment setup (run once)
# This workflow is intended to run in a Workbench environment
# with a managed Cromwell service and GCP authentication enabled.

# Download Cromwell and WOMtool
curl -L -o womtool.jar https://github.com/broadinstitute/cromwell/releases/download/77/womtool-77.jar
curl -L -o cromwell.jar https://github.com/broadinstitute/cromwell/releases/download/77/cromwell-77.jar

# Install SDKMAN (Java dependency manager)
curl -s https://get.sdkman.io | bash

In [ ]:
# This section detects an existing Cromwell service running in the
# user's Workbench workspace and configures cromshell
# to communicate with it.

import os
import subprocess
import json
import requests
from pathlib import Path

def check_for_app(env):
    list_apps_url = f'{env["leonardo_url"]}/api/google/v1/apps/{env["google_project"]}'
    r = requests.get(
        list_apps_url,
        params={
          'includeDeleted': 'false'
        },
        headers = {
            'Authorization': f'Bearer {env["token"]}'
        }
    )
    r.raise_for_status()

    for potential_app in r.json():
        if potential_app['appType'] == 'CROMWELL' and (
                str(potential_app['auditInfo']['creator']) == env['owner_email']
                or str(potential_app['auditInfo']['creator']) == env['user_email']
        ) :
            potential_app_name = potential_app['appName']
            potential_app_status = potential_app['status']

            # We found a CROMWELL app in the correct google project and owned by the user. Now just check the workspace:
            _, workspace_namespace,  proxy_url = get_app_details(env, potential_app_name)
            if workspace_namespace == env['workspace_namespace']:
                return potential_app_name, potential_app_status, proxy_url['cromwell-service']

    return None, None, None

def get_app_details(env, app_name):
    get_app_url = f'{env["leonardo_url"]}/api/google/v1/apps/{env["google_project"]}/{app_name}'
    print('start')
    r = requests.get(
        get_app_url,
        params={
            'includeDeleted': 'true',
            'role': 'creator'
        },
        headers={
            'Authorization': f'Bearer {env["token"]}'
        }
    )
    if r.status_code == 404:
        return 'DELETED', None, None, None
    else:
        r.raise_for_status()
    result_json = r.json()
    custom_environment_variables = result_json['customEnvironmentVariables']
    return result_json['status'], custom_environment_variables['WORKSPACE_NAMESPACE'], result_json.get('proxyUrls')

# Checks that cromshell is installed. Otherwise raises an error.
def validate_cromshell():
    if validate_cromshell_alias():
        print("Found cromshell, please use cromshell")
    elif validate_cromshell_alpha():
        print("Found cromshell-alpha, please use cromshell-alpha")
    else:
        raise Exception("Cromshell is not installed.")

# Checks that cromshell is installed. Otherwise raises an error.
def validate_cromshell_alpha():
    print('Scanning for cromshell 2 alpha...')
    try:
        subprocess.run(['cromshell-alpha', 'version'], capture_output=True, check=True, encoding='utf-8')
    except FileNotFoundError:
        return False
    return True
# Checks that cromshell is installed. Otherwise raises an error.
def validate_cromshell_alias():
    print('Scanning for cromshell 2')
    try:
        subprocess.run(['cromshell', 'version'], capture_output=True, check=True, encoding='utf-8')
    except FileNotFoundError:
        return False
    return True

def configure_cromwell(env, proxy_url):
     print('Updating cromwell config')
     file = f'{str(Path.home())}/.cromshell/cromshell_config.json'
     configuration = {
        'cromwell_server': proxy_url.split("swagger/", 1)[0] if proxy_url else "invalid url",
        'requests_timeout': 5,
        'gcloud_token_email': env['user_email'],
        'referer_header_url': env['leonardo_url']
     }
     with open(file, 'w') as filetowrite:
        filetowrite.write(json.dumps(configuration, indent=2))

def find_app_status(env):
    print(f'Checking status for CROMWELL app')
    app_name, app_status, proxy_url = check_for_app(env)

    configure_cromwell(env, proxy_url)

    if app_name is None:
        print(f'CROMWELL app does not exist. Please create cromwell server from workbench')
    else:
        print(f'app_name={app_name}; app_status={app_status}')
        print(f'Existing CROMWELL app found (app_name={app_name}; app_status={app_status}).')
        exit(1)

def main():
    # Iteration 1: these ENV reads will throw errors if not set.
    env = {
        'workspace_namespace': os.environ['WORKSPACE_NAMESPACE'],
        'workspace_bucket': os.environ['WORKSPACE_BUCKET'],
        'user_email': os.environ.get('PET_SA_EMAIL', default = os.environ['OWNER_EMAIL']),
        'owner_email': os.environ['OWNER_EMAIL'],
        'google_project': os.environ['GOOGLE_PROJECT'],
        'leonardo_url': os.environ['LEONARDO_BASE_URL']
    }

    # Before going any further, check that cromshell2 is installed:
    validate_cromshell()

    # Fetch the token:
    token_fetch_command = subprocess.run(['gcloud', 'auth', 'print-access-token', env['user_email']], capture_output=True, check=True, encoding='utf-8')
    env['token'] = str.strip(token_fetch_command.stdout)

    find_app_status(env)


if __name__ == '__main__':
    main()

In [ ]:
# Create a WDL workflow to run Telogator2 on long-read BAM files
# using scatter parallelization across samples.

wdl_filename = "telogator.wdl"

WDL_content = """version 1.0

task run_telogator2 {
    input {
        File input_file        # Aligned long-read BAM
        String basename        # Sample-derived basename
        Int disk_size          # Disk allocation (GB)
        Int preemptible_tries  # Number of preemptible retries
    }

    command <<<

        set -x  # Enable debug mode to print each command
        echo "Starting task execution..."
        
        echo "Listing files in the working directory:"
        ls -lh
        
        ln -s ~{input_file} ~{basename}.bam

        echo "Running your main script/command..."

        /opt/conda/envs/telogator2/bin/python /opt/telogator2/telogator2.py \
            -i ~{basename}.bam \
            -o ~{basename}/ \
            --winnowmap /opt/conda/envs/telogator2/bin/winnowmap \
            -r X -n X -p X              # Update Telogator2 parameters as needed for dataset (https://github.com/zstephens/telogator2)
            
        echo "Zipping output folder..."
        tar -czvf ~{basename}.tar.gz ~{basename}

        echo "Task completed successfully!"
    >>>

    runtime {
        docker: "niyatij/telogator2_image:latest"
        cpu: "16"
        memory: "32 GB"
        disks: "local-disk ~{disk_size} HDD"
        preemptible: 4
        stderr: "stderr.log"
        stdout: "stdout.log"
    }

    output {
        File results_tar = "~{basename}.tar.gz"
    }
}

workflow telogator2 {
    input {
        Int disk_size
        Int preemptible_tries
        File aligned_bam_file
        String aligned_bam_suffix
    }

    Array[File] aligned_bam = read_lines(aligned_bam_file)

    scatter (aligned_bam in aligned_bam) {
        String basename = sub(sub(aligned_bam, "^.*/", ""), aligned_bam_suffix + "$", "")

        call run_telogator2 {
            input:
                input_file = aligned_bam,
                basename = basename,
                disk_size = disk_size,
                preemptible_tries = preemptible_tries,
        }
    }

    output {
        # Collect all zipped folders
        Array[File] output_telogator2 = run_telogator2.results_tar
    }
}
"""

with open(wdl_filename, 'w') as fp:
    fp.write(WDL_content)

In [ ]:
#Validate WDL
!java -jar womtool-77.jar validate telogator.wdl

In [ ]:
# Workflow inputs
# bam-list.txt contains one google cloud path per line pointing to telomere-2-telomere aligned BAM files (refer to All of Us manifest files for the google cloud paths).
# The file contains no sample metadata or identifiers beyond file paths.
# Make sure the bam-list.txt is copied to your WORKSPACE BUCKET

json_filename = "telogator2_inputs.json"


json_content="""{
    "telogator2.disk_size": XXX, # Set based on BAM size
    "telogator2.preemptible_tries": 3,
    "telogator2.aligned_bam_file": "gs://<WORKSPACE_BUCKET>/bam-list.txt", 
    "telogator2.aligned_bam_suffix": ".bam"
}
"""

fp = open(json_filename, 'w')
fp.write(json_content)
fp.close()

In [ ]:
# Submit the workflow to the managed Cromwell service
!cromshell submit telogator.wdl telogator2_inputs.json